In [43]:
from openai import OpenAI
from pydantic import BaseModel
import functions.prompts as prompts
import functions.llm as llm
import functions.fetch as fetch
import functions.triplets as triplets
import pandas as pd
import json
from openai.lib._parsing._completions import type_to_response_format_param

import importlib
importlib.reload(llm)
importlib.reload(prompts)
importlib.reload(fetch)
importlib.reload(triplets)

client = OpenAI()

In [44]:
class Subject(BaseModel):
   type: str # human | ai | co ONLY
   subtype: str # generative | student
   feature: str # creativity | explaination | #trust
   
class Triplet(BaseModel):
   cause: Subject
   relationship: str
   effect: Subject

In [45]:
df = pd.read_csv("./data/findings/findings.csv")

reqs = []
for i, row in df.iterrows():
    reqs.append(llm.wrap(f"{i}:{row['paper-id']}", {
        "model": "o3-mini",
        "messages": [{
            "role": "system",
            "content": prompts.triplets
        }, {
            "role": "user",
            "content": row['finding']
        }],
        "response_format": type_to_response_format_param(Triplet)
    }))

req_path = "data/triplets/req.jsonl"
# llm.gen_batch_jsonl(req_path, reqs)

In [46]:
# print(f"File ID: {batch['batch_file']}")
# print(f"Batch ID: {batch['batch_object']}")
# 
# fetch.save(f"File ID: {batch['batch_file']} \nBatch ID: {batch['batch_object']}", "data/triplets/req_file_id.txt")

In [47]:
res_path = "data/triplets/res.jsonl"
# output_file_id = llm.get_batch_result(client, batch['batch_object'], res_path)

In [48]:
import pandas as pd

f =  open(res_path, "r")
res = f.read()
f.close()

df = pd.DataFrame(columns=["paper-id", "cause", "relation", "effect"])
df_keys = []

for i in res.split("\n")[:-1]:
    line = json.loads(i)
    id = line['custom_id']
    try:
        content = Triplet.model_validate_json(line['response']['body']['choices'][0]['message']['content'])
        df.loc[len(df)] = [id, triplets.parse_subject(content.cause), str(content.relationship), triplets.parse_subject(content.effect)]
        keys = [f"{content.cause.type}:{content.cause.subtype}:{content.cause.feature}", content.relationship, content.effect]
        df_keys += triplets.parse_key_list(content.cause)
        if("human|user" in triplets.parse_key_list(content.cause)):
            print(content.cause)
        df_keys += triplets.parse_key_list(content.effect)
        if("human|user" in triplets.parse_key_list(content.cause)):
            print(content.cause)

    except Exception as e:
        print(f"Error processing line ({id})")
    
df_keys = set(df_keys)
fetch.save("\n".join([*df_keys]), "data/embeddings/keys.txt")
df.to_csv("data/triplets/triplets.csv", index=False)

Error processing line (1267:paper-894)
